In [ ]:
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
with open('v2_OpenEnded_mscoco_train2014_questions.json', 'r') as f:
    data = json.load(f)

In [ ]:
df_q = pd.DataFrame(data['questions'])


In [ ]:
with open('v2_mscoco_train2014_annotations.json', 'r') as f:
    data = json.load(f)

In [ ]:
df_a = pd.DataFrame(data['annotations'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("anananan116/TinyVLM")

In [ ]:
df_a.iloc[0]

In [ ]:
df_q.iloc[0]

In [ ]:
SYSTEM_PROMPT = "You are a powerful visual assistant."

In [ ]:
conversation = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": df_q.iloc[0]['question'] + "<IMGPLH>"},
    {"role": "assistant", "content": df_a.iloc[0]['multiple_choice_answer']}
]

In [ ]:
conversation

In [ ]:
conversation = tokenizer.apply_chat_template(conversation, tokenize=False)

In [ ]:
image = "VQA/" + str(df_q.iloc[0]['image_id'])

In [ ]:
temp = {
    "text": [conversation,],
    "image1": [image]
}

In [ ]:
df = pd.DataFrame(temp)

In [ ]:
df

In [ ]:
df_q.head()

In [ ]:
df_a.head()

In [ ]:
# df_a = df_a.drop('question_id', axis=1)
df_QA =  pd.merge(df_q, df_a, on=['question_id','image_id'], how='inner')
df_QA.head(15)

In [ ]:
df_QA['multiple_choice_answer'] = df_QA.apply(lambda col: "It's " + col['multiple_choice_answer'] if col['answer_type'] == 'other' else col['multiple_choice_answer'], axis = 1)
df_QA['multiple_choice_answer']

In [ ]:
df_QA['question'] = df_QA['question'].map(lambda x: '<user>' + x+'</user>')
df_QA['multiple_choice_answer'] = df_QA['multiple_choice_answer'].map(lambda x: '<assist>' + x+'</assist>')
df_QA['system_prompt'] = "<sys>You are a useful Visual Assitant</sys>"


In [ ]:
df_QA['prompt'] = df_QA.apply(lambda row: f"{row['system_prompt']}{row['question']}{row['multiple_choice_answer']}", axis=1)

In [ ]:
df_final = df_QA[['image_id','prompt']]
df_final.head()

In [ ]:
df_final.to_csv('output.csv', index=False)